In [1]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [2]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 


In [22]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 5.0 MB/s 


In [3]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory
from openpyxl import load_workbook

In [68]:
item=pd.read_excel(open('Birchbox_data.xlsx', 'rb'),
              sheet_name='item info')
item.head()

,Item,Category,Inventory,Sample Cost,Margin if Purchase,PurchaseProb_Seg_0,PurchaseProb_Seg_1,PurchaseProb_Seg_2,PurchaseProb_Seg_3,PurchaseProb_Seg_4,PurchaseProb_Seg_5,PurchaseProb_Seg_6,PurchaseProb_Seg_7,PurchaseProb_Seg_8,PurchaseProb_Seg_9,PurchaseProb_Seg_10,PurchaseProb_Seg_11,PurchaseProb_Seg_12,PurchaseProb_Seg_13
0,0,6,94,7.5,51.475382,0.090036,0.100748,0.248534,0.288539,0.104476,0.112507,0.091812,0.120041,0.087837,0.089800,0.098098,0.092363,0.113431,0.115416
1,1,7,107,3.2,26.316932,0.288539,0.093838,0.122771,0.088250,0.100276,0.100276,0.096180,0.109183,0.094151,0.101234,0.092085,0.103327,0.092646,0.099370
2,2,3,119,0.8,3.772008,0.098511,0.248534,0.086859,0.109183,0.101734,0.097696,0.107713,0.106349,0.092363,0.090036,0.124267,0.087636,0.098511,0.088675
3,3,2,153,9.5,50.397349,0.101734,0.087636,0.091545,0.088250,0.086859,0.116483,0.087049,0.182048,0.092646,0.147708,0.205559,0.088042,0.087837,0.182048
4,4,7,176,1.0,5.267752,0.100276,0.092935,0.099817,0.092085,0.173718,0.131383,0.094798,0.182048,0.087437,0.107713,0.088042,0.118790,0.093838,0.090770


In [69]:
cust=pd.read_excel(open('Birchbox_data.xlsx', 'rb'),
              sheet_name='cust info')
cust.head()

,Cust #,Seg,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,Item 10,Item 11,Item 12,Item 13,Item 14,Item 15,Item 16,Item 17,Item 18,Item 19,Item 20,Item 21,Item 22,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32,Item 33,Item 34,Item 35,Item 36,Item 37,...,Item 71,Item 72,Item 73,Item 74,Item 75,Item 76,Item 77,Item 78,Item 79,Item 80,Item 81,Item 82,Item 83,Item 84,Item 85,Item 86,Item 87,Item 88,Item 89,Item 90,Item 91,Item 92,Item 93,Item 94,Item 95,Item 96,Item 97,Item 98,Item 99,Item 100,Item 101,Item 102,Item 103,Item 104,Item 105,Item 106,Item 107,Item 108,Item 109,Item 110
0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1
3,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,6,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [70]:
category= item['Category'].values.tolist() #indexed by item, i 
inventory=item['Inventory'].values.tolist()
sample_cost=item['Sample Cost'].values.tolist()
mip=item['Margin if Purchase'].values.tolist()
prob=item.iloc[:,5:].values.tolist() #indexed by item, i row-wise and segment columnwise 

In [71]:
segment= cust['Seg'].values.tolist() #indexed by customer, j
past_send=cust.iloc[:,2:].values.tolist() #indexed by customer, j row-wise and item, i columnwise 


In [74]:
#Part 1
#Declare a model 
model=ConcreteModel()
num_item=len(item) #indexed by i
num_cust=len(cust) #indexed by j 

#Declare DVs
model.x=Var(range(num_cust), range(num_item), domain=Binary)

#Set constraints
#5 items need to be sent to each customer
model.constraint1=ConstraintList()
for j in range(num_cust): 
   model.constraint1.add(expr= sum(model.x[j,i] for i in range(num_item)) == 5)

#You can't run out of inventory 
model.constraint2=ConstraintList()
for i in range(num_item):
  model.constraint2.add(expr= sum(model.x[j,i] for j in range(num_cust)) <= inventory[i])

#You can't send a customer a sample they have recieved 
model.constraint3=ConstraintList()
for i in range(num_item):
  for j in range(num_cust):
    model.constraint3.add(expr= model.x[j,i] + past_send[j][i]  <= 1)


#Define an objective
total_cost= None
total_margin=None 

for j in range(num_cust):
  total_cost+=sum(model.x[j,i]*sample_cost[i] for i in range(num_item))

for j in range(num_cust):
  k=segment[j]
  total_margin+=sum(model.x[j,i]*prob[i][k]*mip[i] for i in range(num_item))

model.Objective=Objective(expr= total_margin-total_cost, sense=maximize)

#Solve
opt=SolverFactory("glpk")
opt.options["mipgap"]=0.001 #tolerance
results=opt.solve(model)

#Save results into lists and print profit 
cust_sent=[]
item_sent = [[] for i in range(num_cust)]
for j in range(num_cust):
  cust_sent.append(j)
  for i in range(num_item):
    if model.x[j,i]() == 1.0:
      item_sent[j].append(i)

profit=model.Objective()
print("Profit is", profit)

Profit is 15957.657040005404


In [75]:
solution=pd.DataFrame()
solution['Customers']=cust_sent
solution['Items to be sent']=item_sent
solution.head()

,Customers,Items to be sent
0,0,"[7, 9, 14, 34, 41]"
1,1,"[24, 51, 67, 94, 105]"
2,2,"[14, 30, 31, 41, 58]"
3,3,"[12, 22, 28, 55, 109]"
4,4,"[6, 22, 33, 60, 76]"


In [76]:
#Save the solution as a new sheet in our excel file 
path = "/content/Birchbox_data.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

solution.to_excel(writer, sheet_name = 'solutionPart1')
writer.save()
writer.close()

In [77]:
#Part 2
#Update the inventory 
for j in range(num_cust):
  for i in range(num_item):
    if model.x[j,i]() == 1.0:
      inventory[i]-=1

#Update past_send matrix
for j in range(num_cust):
  for i in range(num_item):
    if model.x[j,i]() == 1.0:
      past_send[j][i]+=1

In [78]:
#Declare a model 
model=ConcreteModel()
num_item=len(item) #indexed by i
num_cust=len(cust) #indexed by j 

#Declare DVs
model.x=Var(range(num_cust), range(num_item), domain=Binary)

#Set constraints
#5 items need to be sent to each customer
model.constraint1=ConstraintList()
for j in range(num_cust): 
   model.constraint1.add(expr= sum(model.x[j,i] for i in range(num_item)) == 5)

#You can't run out of inventory 
model.constraint2=ConstraintList()
for i in range(num_item):
  model.constraint2.add(expr= sum(model.x[j,i] for j in range(num_cust)) <= inventory[i])

#You can't send a customer a sample they have recieved 


#Define an objective
total_cost= None
total_margin=None 

for j in range(num_cust):
  total_cost+=sum(model.x[j,i]*sample_cost[i] for i in range(num_item))

for j in range(num_cust):
  k=segment[j]
  total_margin+=sum(model.x[j,i]*prob[i][k]*mip[i] for i in range(num_item))

model.Objective=Objective(expr= total_margin-total_cost, sense=maximize)

#Solve
opt=SolverFactory("glpk")
opt.options["mipgap"]=0.001 #tolerance
results=opt.solve(model)

#Save results into lists
cust_sent2=[]
item_sent2 = [[] for i in range(num_cust)]
for j in range(num_cust):
  cust_sent2.append(j)
  for i in range(num_item):
    if model.x[j,i]() == 1.0:
      item_sent2[j].append(i)

profit2=model.Objective()
print("Profit is", profit2)

Profit is 12556.692033782416


In [79]:
solution2=pd.DataFrame()
solution2['Customers']=cust_sent2
solution2['Items to be sent']=item_sent2
solution2.head()

,Customers,Items to be sent
0,0,"[8, 9, 41, 68, 88]"
1,1,"[24, 51, 67, 78, 94]"
2,2,"[4, 29, 31, 58, 88]"
3,3,"[1, 28, 55, 107, 109]"
4,4,"[6, 22, 33, 60, 87]"


In [80]:
path = "/content/Birchbox_data.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

solution2.to_excel(writer, sheet_name = 'solutionPart2')
writer.save()
writer.close()